# The Need for The Feed 📱😵‍💫

Helin Yilmaz, Maya Alhashem, Nory Arroyo, Oviya Adhan, Vanessa Navarro

DATASCI 209 Data Visualization

Final Project Exploratory Data Visualization

### Import Libraries

In [1]:
import pandas as pd
import numpy as np
import altair as alt

### Import Data

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
mental_df_clean = pd.read_csv('/content/drive/MyDrive/209 Final Project/mental_health_clean.csv')
social_df_clean = pd.read_csv('/content/drive/MyDrive/209 Final Project/social_media_clean.csv')


In [4]:
print(f"Mental health data: {mental_df_clean.shape}")
print(f"Social media data: {social_df_clean.shape}")

Mental health data: (24998, 17)
Social media data: (449, 35)


In [5]:
mental_df_clean.head()

,User_ID,Age,Gender,Occupation,Country,Mental_Health_Condition,Severity,Consultation_History,Stress_Level,Sleep_Hours,Work_Hours,Physical_Activity_Hours,Social_Media_Usage,Diet_Quality,Smoking_Habit,Alcohol_Consumption,Medication_Usage
0,2,48,Male,Engineering,Other,No,Low,No,Low,6.8,74,2,3.4,Unhealthy,Heavy Smoker,Social Drinker,No
1,4,30,Non-binary,Engineering,Australia,No,Medium,No,Low,6.9,57,4,5.4,Average,Regular Smoker,Regular Drinker,No
2,6,34,Non-binary,Healthcare,Australia,No,High,No,Low,5.1,74,4,3.6,Unhealthy,Regular Smoker,Social Drinker,Yes
3,7,24,Non-binary,Engineering,India,Yes,Medium,Yes,High,7.1,35,4,6.0,Average,Non-Smoker,Non-Drinker,Yes
4,9,53,Male,Other,USA,No,Low,No,Low,4.8,62,3,5.7,Unhealthy,Occasional Smoker,Regular Drinker,Yes


In [6]:
mental_df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24998 entries, 0 to 24997
Data columns (total 17 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   User_ID                  24998 non-null  int64  
 1   Age                      24998 non-null  int64  
 2   Gender                   24998 non-null  object 
 3   Occupation               24998 non-null  object 
 4   Country                  24998 non-null  object 
 5   Mental_Health_Condition  24998 non-null  object 
 6   Severity                 24998 non-null  object 
 7   Consultation_History     24998 non-null  object 
 8   Stress_Level             24998 non-null  object 
 9   Sleep_Hours              24998 non-null  float64
 10  Work_Hours               24998 non-null  int64  
 11  Physical_Activity_Hours  24998 non-null  int64  
 12  Social_Media_Usage       24998 non-null  float64
 13  Diet_Quality             24998 non-null  object 
 14  Smoking_Habit         

In [7]:
social_df_clean.head()

,timestamp,age,gender,relationship_status,occupation_status,org_affiliation,sm_usage,sm_platforms,sm_time_daily,sm_without_purpose_freq,...,University,Discord,Facebook,Instagram,Pinterest,Reddit,Snapchat,TikTok,Twitter,YouTube
0,2022-04-18 19:18:47,21,Male,In a relationship,University Student,University,Yes,"Facebook, Twitter, Instagram, YouTube, Discord...",Between 2 and 3 hours,5,...,True,True,True,True,False,True,False,False,True,True
1,2022-04-18 19:19:28,21,Female,Single,University Student,University,Yes,"Facebook, Twitter, Instagram, YouTube, Discord...",More than 5 hours,4,...,True,True,True,True,False,True,False,False,True,True
2,2022-04-18 19:25:59,21,Female,Single,University Student,University,Yes,"Facebook, Instagram, YouTube, Pinterest",Between 3 and 4 hours,3,...,True,False,True,True,True,False,False,False,False,True
3,2022-04-18 19:29:43,21,Female,Single,University Student,University,Yes,"Facebook, Instagram",More than 5 hours,4,...,True,False,True,True,False,False,False,False,False,False
4,2022-04-18 19:33:31,21,Female,Single,University Student,University,Yes,"Facebook, Instagram, YouTube",Between 2 and 3 hours,3,...,True,False,True,True,False,False,False,False,False,True


In [ ]:
social_df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 449 entries, 0 to 448
Data columns (total 35 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   timestamp                   449 non-null    object
 1   age                         449 non-null    int64 
 2   gender                      449 non-null    object
 3   relationship_status         449 non-null    object
 4   occupation_status           449 non-null    object
 5   org_affiliation             449 non-null    object
 6   sm_usage                    449 non-null    object
 7   sm_platforms                449 non-null    object
 8   sm_time_daily               449 non-null    object
 9   sm_without_purpose_freq     449 non-null    int64 
 10  sm_distraction_freq         449 non-null    int64 
 11  sm_restless_if_unused       449 non-null    int64 
 12  distractibility             449 non-null    int64 
 13  worried                     449 non-null    int64 

In [ ]:
# Create the composite 'stress_anxiety_level' column to use in the viz
social_df_clean['stress_anxiety_level'] = (
    social_df_clean['worried'] +
    social_df_clean['concentration_difficulty'] +
    social_df_clean['sm_comparison_feelings'] +
    social_df_clean['depressed_freq'] +
    # social_df_clean['sleep_issues']
)

In [ ]:
#  Standardized Time Data
def parse_sm_time(time_str):
    if pd.isna(time_str):
        return None
    if "More than 5 hours" in time_str:
        return 5.5
    elif "Between 4 and 5 hours" in time_str:
        return 4.5
    elif "Between 3 and 4 hours" in time_str:
        return 3.5
    elif "Between 2 and 3 hours" in time_str:
        return 2.5
    elif "Between 1 and 2 hours" in time_str:
        return 1.5
    elif "Less than an Hour" in time_str:
        return 0.5
    else:
        return None

social_df_clean['sm_time_daily_num'] = social_df_clean['sm_time_daily'].apply(parse_sm_time)


In [ ]:
# Define the explicit order for 'sm_time_daily'
sm_time_order = [
    "Less than an Hour",
    "Between 1 and 2 hours",
    "Between 2 and 3 hours",
    "Between 3 and 4 hours",
    "Between 4 and 5 hours",
    "More than 5 hours"
]

In [ ]:
# Chart 1
chart1 = alt.Chart(social_df_clean).mark_bar().encode(
    x=alt.X('sm_time_daily:N',
            sort=sm_time_order,
            title='Daily Social Media Usage',
            axis=alt.Axis(titlePadding=15)),
    y=alt.Y('mean(stress_anxiety_level):Q',
            title='Average Stress/Anxiety Score',
            axis=alt.Axis(titlePadding=15)),
    color=alt.Color('sm_time_daily:N', sort=sm_time_order, scale=alt.Scale(scheme='oranges')),
    tooltip=[
        alt.Tooltip('count():Q', title='Number of Respondents'),
        alt.Tooltip('mean(stress_anxiety_level):Q', title='Avg Stress/Anxiety', format='.2f')
    ]
).properties(
    width=600,
    height=400,
    title='Average Stress/Anxiety Score by Daily Social Media Usage'
).configure_axis(
    labelFontSize=14,
    titleFontSize=16
).configure_title(
    fontSize=22,
    offset=20
).configure_legend(
    labelFontSize=14,
    titleFontSize=16
)
chart1
# chart1.save('chart1.json')

alt.Chart(...)

In [ ]:
# Chart 2
filtered_df = social_df_clean.dropna(subset=['sm_time_daily', 'gender'])
age_bins_3 = [0, 30, 50, 100]
age_labels_3 = ['Under 30', '30-49', '50+']
filtered_df['age_group_3'] = pd.cut(filtered_df['age'], bins=age_bins_3, labels=age_labels_3, right=False)

bars = alt.Chart(filtered_df).mark_bar().encode(
    x=alt.X('sm_time_daily:N', title='Daily Social Media Usage', sort=sm_time_order, axis=alt.Axis(titlePadding=25)),
    y=alt.Y('mean(stress_anxiety_level):Q', title='Average Stress/Anxiety Score', axis=alt.Axis(titlePadding=25)),
    color=alt.Color('gender:N', title='Gender', scale=alt.Scale(scheme='tableau10')),
    tooltip=[
        alt.Tooltip('gender:N'),
        alt.Tooltip('count():Q', title='Respondents'),
        alt.Tooltip('mean(stress_anxiety_level):Q', title='Avg Stress/Anxiety', format='.2f')
    ]
).properties(
    width=250,
    height=300
)
error_bars = alt.Chart(filtered_df).mark_errorbar(extent='ci').encode(
    x=alt.X('sm_time_daily:N', sort=sm_time_order, axis=alt.Axis(titlePadding=25)),
    y=alt.Y('stress_anxiety_level:Q', axis=alt.Axis(titlePadding=25)),
    color=alt.Color('gender:N')
).properties(
    width=250,
    height=300
)
combined = bars + error_bars

chart = combined.facet(
    column=alt.Column('age_group_3:N', title='Age Group')
).properties(
    title='Average Stress/Anxiety by Social Media Usage, Gender, and Age Group'
).configure_axis(
    labelFontSize=12,
    titleFontSize=14
).configure_header(
    titleFontSize=15,
    labelFontSize=13
).configure_title(
    fontSize=20,
    anchor='middle',
    offset=20
).configure_legend(
    labelFontSize=12,
    titleFontSize=14
)
chart
# chart.save('chart2.json')


alt.FacetChart(...)

In [ ]:
print("Data type:", social_df_clean['sleep_issues'].dtype)
print("Missing values:", social_df_clean['sleep_issues'].isnull().sum())
print("Unique values and counts:\n", social_df_clean['sleep_issues'].value_counts(dropna=False))
print("\nSummary statistics:\n", social_df_clean['sleep_issues'].describe())


Data type: int64
Missing values: 0
Unique values and counts:
 sleep_issues
5    115
4    103
2     87
1     78
3     66
Name: count, dtype: int64

Summary statistics:
 count    449.000000
mean       3.200445
std        1.451569
min        1.000000
25%        2.000000
50%        3.000000
75%        5.000000
max        5.000000
Name: sleep_issues, dtype: float64


In [ ]:
filtered_df = social_df_clean.dropna(subset=['sm_time_daily', 'gender', 'Instagram', 'Facebook', 'TikTok', 'Twitter'])
age_bins_3 = [0, 30, 50, 100]
age_labels_3 = ['Under 30', '30-49', '50+']
filtered_df['age_group_3'] = pd.cut(filtered_df['age'], bins=age_bins_3, labels=age_labels_3, right=False)

# Create a new column 'uses_any_platform' that's True if they use any of the 4 platforms
filtered_df['uses_any_platform'] = (
    filtered_df[['Instagram', 'Facebook', 'TikTok', 'Twitter']]
    .any(axis=1)
).map({True: 'Uses Instagram/Facebook/TikTok/Twitter', False: 'Does not use any'})

bars = alt.Chart(filtered_df).mark_bar().encode(
    x=alt.X('sm_time_daily:N', title='Daily Social Media Usage', sort=sm_time_order, axis=alt.Axis(titlePadding=25)),
    y=alt.Y('mean(stress_anxiety_level):Q', title='Average Stress Score', axis=alt.Axis(titlePadding=25)),
    color=alt.Color('gender:N', title='Gender', scale=alt.Scale(scheme='tableau10')),
    tooltip=[
        alt.Tooltip('gender:N'),
        alt.Tooltip('count():Q', title='Respondents'),
        alt.Tooltip('mean(stress_anxiety_level):Q', title='Avg Stress/Anxiety', format='.2f')
    ]
).properties(
    width=250,
    height=250
)

error_bars = alt.Chart(filtered_df).mark_errorbar(extent='ci').encode(
    x=alt.X('sm_time_daily:N', sort=sm_time_order, axis=alt.Axis(titlePadding=25)),
    y=alt.Y('stress_anxiety_level:Q', axis=alt.Axis(titlePadding=25)),
    color=alt.Color('gender:N')
).properties(
    width=250,
    height=250
)

combined = bars + error_bars

chart = combined.facet(
    column=alt.Column('age_group_3:N', title='Age Group'),
    row=alt.Row('uses_any_platform:N', title='Platform Usage')
).properties(
    title='Average Stress/Anxiety by Social Media Usage, Gender, Age Group, and Platform Use'
).configure_axis(
    labelFontSize=12,
    titleFontSize=14
).configure_header(
    titleFontSize=15,
    labelFontSize=13
).configure_title(
    fontSize=20,
    anchor='middle',
    offset=20
).configure_legend(
    labelFontSize=12,
    titleFontSize=14
)

chart
chart.save('chart3.json')


In [ ]:
base = alt.Chart(filtered_df).mark_rect().encode(
    x=alt.X('sm_time_daily_num:Q', bin=alt.Bin(maxbins=10), title='Social Media Time (hours)'),
    y=alt.Y('worried:Q', bin=alt.Bin(maxbins=10), title='Worry Score'),
    color=alt.Color('count():Q', scale=alt.Scale(scheme='viridis'), title='Respondent Count'),
    tooltip=[alt.Tooltip('count()', title='Count')]
).properties(
    width=350,
    height=350
)

chart_heat = base.facet(
    column=alt.Column('uses_any_platform:N', title='Uses Any Platform')
).properties(
    title='Heatmap of Social Media Usage vs Worry Score, Faceted by Platform Use'
).configure_title(
    fontSize=24,
    anchor='middle',
    offset=30
).configure_axis(
    labelFontSize=16,
    titleFontSize=18
).configure_header(
    labelFontSize=18,
    titleFontSize=20
)

chart_heat
# chart_heat.save('chart_heat.json')


alt.FacetChart(...)

In [ ]:
scatter_stress = alt.Chart(social_df_clean).mark_circle(size=60, opacity=0.6).encode(
    x=alt.X('sm_time_daily_num:Q', title='Daily Social Media Usage (hours)'),
    y=alt.Y('stress_anxiety_level:Q', title='Stress/Anxiety Score'),
    tooltip=[
        alt.Tooltip('sm_time_daily_num:Q', title='Daily Social Media Time', format='.1f'),
        alt.Tooltip('stress_anxiety_level:Q', title='Stress/Anxiety Score', format='.2f')
    ]
).properties(
    width=600,
    height=400,
    title='Scatterplot of Stress/Anxiety Score vs Daily Social Media Time'
).configure_axis(
    labelFontSize=14,
    titleFontSize=16
).configure_title(
    fontSize=20,
    anchor='middle',
    offset=10
)

scatter_stress
# scatter_stress.save('scatter_stress.json')


alt.Chart(...)